In [1]:
import requests

url = "https://api.elections.kalshi.com/trade-api/v2/exchange/status"

response = requests.get(url)

print(response.text)

{"exchange_active":true,"exchange_estimated_resume_time":null,"trading_active":true}


In [2]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend

def load_private_key_from_file(file_path):
    with open(file_path, "rb") as key_file:
        private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None,  # or provide a password if your key is encrypted
            backend=default_backend()
        )
    return private_key

In [3]:
import base64
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding, rsa
from cryptography.exceptions import InvalidSignature

def sign_pss_text(private_key: rsa.RSAPrivateKey, text: str) -> str:
    message = text.encode('utf-8')
    try:
        signature = private_key.sign(
            message,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.DIGEST_LENGTH
            ),
            hashes.SHA256()
        )
        return base64.b64encode(signature).decode('utf-8')
    except InvalidSignature as e:
        raise ValueError("RSA sign PSS failed") from e

In [4]:
import requests
import datetime
from pathlib import Path


current_time = datetime.datetime.now()
timestamp = current_time.timestamp()
current_time_milliseconds = int(timestamp * 1000)
timestampt_str = str(current_time_milliseconds)

private_key = load_private_key_from_file('src/secrets/kalshi_private_key.pem')
api_key = Path('src/secrets/kalshi-key-2.key').read_text().strip()

method = "GET"
base_url = 'https://api.elections.kalshi.com'
path='/trade-api/v2/portfolio/balance'

# Strip query parameters from path before signing
path_without_query = path.split('?')[0]
msg_string = timestampt_str + method + path_without_query
sig = sign_pss_text(private_key, msg_string)

headers = {
    'KALSHI-ACCESS-KEY': api_key,
    'KALSHI-ACCESS-SIGNATURE': sig,
    'KALSHI-ACCESS-TIMESTAMP': timestampt_str
}

response = requests.get(base_url + path, headers=headers)

print(response.text)

FileNotFoundError: [Errno 2] No such file or directory: 'src/secrets/kalshi_private_key.pem'

In [5]:
import requests

def get_all_markets(series_ticker, endpoint):
    """Fetch all markets for a series, handling pagination"""
    all_markets = []
    cursor = None
    base_url = f"https://api.elections.kalshi.com/trade-api/v2/{endpoint}"

    while True:
        # Build URL with cursor if we have one
        url = f"{base_url}?series_ticker={series_ticker}&limit=100"
        if cursor:
            url += f"&cursor={cursor}"

        response = requests.get(url)
        data = response.json()

        # Add markets from this page
        all_markets.extend(data[endpoint])

        # Check if there are more pages
        cursor = data.get('cursor')
        if not cursor:
            break

        print(f"Fetched {len(data[endpoint])} {endpoint}, total: {len(all_markets)}")

    return all_markets

# Example usage
page = "events"
markets = get_all_markets("KXFEDCHAIRNOM-29", page)
print(f"Total {page} found: {len(markets)}")

Total events found: 0


In [6]:

url = "https://api.elections.kalshi.com/trade-api/v2/markets?limit=10&status=open"
b = requests.get(url)

In [7]:
import requests

# Get orderbook for a specific market
market_ticker = "KXFEDCHAIRNOM-29-KW"
url = f"https://api.elections.kalshi.com/trade-api/v2/markets/{market_ticker}/orderbook"

response = requests.get(url)
orderbook_data = response.json()

In [8]:
orderbook_data

{'orderbook': {'no': [[1, 352205],
   [2, 244483],
   [3, 114162],
   [4, 268956],
   [5, 13401]],
  'no_dollars': [['0.0100', 352205],
   ['0.0200', 244483],
   ['0.0300', 114162],
   ['0.0400', 268956],
   ['0.0500', 13401]],
  'yes': [[1, 115130],
   [2, 107762],
   [3, 20005],
   [4, 101000],
   [5, 30250],
   [6, 195],
   [8, 81],
   [9, 90],
   [10, 5985],
   [11, 527],
   [15, 20],
   [16, 1000],
   [17, 1000],
   [20, 102],
   [25, 1],
   [30, 10],
   [50, 6856],
   [51, 10000],
   [52, 350300],
   [53, 1037],
   [55, 100100],
   [56, 10000],
   [80, 600],
   [81, 15],
   [85, 250],
   [86, 299],
   [89, 30015],
   [90, 72023],
   [91, 1733],
   [92, 47355],
   [93, 79267],
   [94, 648278]],
  'yes_dollars': [['0.0100', 115130],
   ['0.0200', 107762],
   ['0.0300', 20005],
   ['0.0400', 101000],
   ['0.0500', 30250],
   ['0.0600', 195],
   ['0.0800', 81],
   ['0.0900', 90],
   ['0.1000', 5985],
   ['0.1100', 527],
   ['0.1500', 20],
   ['0.1600', 1000],
   ['0.1700', 1000],
   

In [9]:
import requests

url = "https://api.elections.kalshi.com/trade-api/v2/exchange/announcements"

response = requests.get(url)

print(response.text)

{"announcements":[]}


In [10]:
import requests

url = "https://api.elections.kalshi.com/trade-api/v2/series/fee_changes"

response = requests.get(url)

print(response.text)

{"series_fee_change_arr":[]}


In [15]:
import requests

url = "https://api.elections.kalshi.com/trade-api/v2/exchange/schedule"

response = requests.get(url)

print(response.text)

{"schedule":{"maintenance_windows":[],"standard_hours":[{"end_time":"2200-12-01T00:00:00Z","friday":[{"close_time":"00:00","open_time":"00:00"}],"monday":[{"close_time":"00:00","open_time":"00:00"}],"saturday":[{"close_time":"00:00","open_time":"00:00"}],"start_time":"2024-12-01T00:00:00Z","sunday":[{"close_time":"00:00","open_time":"00:00"}],"thursday":[{"close_time":"03:00","open_time":"00:00"},{"close_time":"00:00","open_time":"05:00"}],"tuesday":[{"close_time":"00:00","open_time":"00:00"}],"wednesday":[{"close_time":"00:00","open_time":"00:00"}]}]}}


In [16]:
# https://api.elections.kalshi.com/trade-api/v2/markets


In [13]:
import requests
from datetime import datetime, timezone

BASE = "https://api.elections.kalshi.com/trade-api/v2"


def fetch_all_open_markets(limit=1000):
    markets = []
    cursor = None

    while True:
        params = {"status": "open", "limit": limit}
        if cursor:
            params["cursor"] = cursor

        r = requests.get(f"{BASE}/markets", params=params, timeout=30)
        r.raise_for_status()
        data = r.json()

        markets.extend(data.get("markets", []))
        cursor = data.get("cursor")

        if not cursor:
            break

    return markets


def export_open_markets_json():
    raw_markets = fetch_all_open_markets()

    out = {
        "metadata": {
            "generated_at": datetime.now(timezone.utc).isoformat(),
            "market_count": len(raw_markets),
            "event_count": 0,
            "series_count": 0,
        },
        "series": {},
    }

    for m in raw_markets:
        series_id = m.get("series_id", "UNKNOWN_SERIES")
        event_id = m.get("event_id", "UNKNOWN_EVENT")

        series = out["series"].setdefault(series_id, {
            "series_id": series_id,
            "events": {}
        })

        events = series["events"]
        event = events.setdefault(event_id, {
            "event_id": event_id,
            "event_title": m.get("event_title"),
            "close_ts": m.get("close_ts"),
            "markets": []
        })

        event["markets"].append({
            "ticker": m.get("ticker"),
            "title": m.get("title"),
            "status": m.get("status"),
            "yes_ask": m.get("yes_ask"),
            "no_ask": m.get("no_ask"),
            "volume": m.get("volume"),
            "open_interest": m.get("open_interest"),
            "close_ts": m.get("close_ts"),
        })

    out["metadata"]["series_count"] = len(out["series"])
    out["metadata"]["event_count"] = sum(
        len(s["events"]) for s in out["series"].values()
    )

    return out


In [14]:
import json
import datetime as dt

now = dt.datetime.now().date()
data = export_open_markets_json()

with open(f"kalshi_open_markets_{now}.json", "w") as f:
    json.dump(data, f, indent=2)


KeyboardInterrupt: 

In [7]:
from src.data.db_export_open_markets import fetch_open_markets

def test_fetch_open_markets():
    markets = fetch_open_markets()
    assert isinstance(markets, list)
    assert len(markets) > 0
    assert all(isinstance(m, dict) for m in markets)

test_fetch_open_markets()

Fetched page 1, +1000 markets (total=1000)
Fetched page 2, +1000 markets (total=2000)
Fetched page 3, +1000 markets (total=3000)
Fetched page 4, +1000 markets (total=4000)
Fetched page 5, +1000 markets (total=5000)
Fetched page 6, +1000 markets (total=6000)
Fetched page 7, +1000 markets (total=7000)
Fetched page 8, +1000 markets (total=8000)
Fetched page 9, +1000 markets (total=9000)
Fetched page 10, +1000 markets (total=10000)
Fetched page 11, +1000 markets (total=11000)
Fetched page 12, +1000 markets (total=12000)
Fetched page 13, +1000 markets (total=13000)
Fetched page 14, +1000 markets (total=14000)
Fetched page 15, +1000 markets (total=15000)
Fetched page 16, +1000 markets (total=16000)
Fetched page 17, +1000 markets (total=17000)
Fetched page 18, +1000 markets (total=18000)
Fetched page 19, +1000 markets (total=19000)
Fetched page 20, +1000 markets (total=20000)
Fetched page 21, +1000 markets (total=21000)
Fetched page 22, +1000 markets (total=22000)
Fetched page 23, +1000 marke

In [3]:
import json
import time
from datetime import datetime, timezone
from typing import Any, Dict, List

import requests

GAMMA_BASE = "https://gamma-api.polymarket.com"


def _now_iso() -> str:
    return datetime.now(timezone.utc).isoformat()


def _maybe_parse_json_string(val: Any) -> Any:
    if isinstance(val, str):
        s = val.strip()
        if (s.startswith("[") and s.endswith("]")) or (s.startswith("{") and s.endswith("}")):
            try:
                return json.loads(s)
            except Exception:
                return val
    return val


def _get_json(session: requests.Session, url: str, params: Dict[str, Any], timeout: int = 30) -> Any:
    r = session.get(url, params=params, timeout=timeout)
    # If you want to see the exact error payload when debugging:
    # print("URL:", r.url, "STATUS:", r.status_code, "BODY:", r.text[:500])
    r.raise_for_status()
    return r.json()


def fetch_all_active_events(
    limit: int = 100,
    order: str = "volume24hr",   # <-- IMPORTANT: matches the schema field name
    ascending: str = "false",
    max_retries: int = 5,
) -> List[Dict[str, Any]]:
    """
    Fetch ALL active events using offset pagination:
      GET /events?active=true&closed=false&limit=...&offset=...

    Uses order=volume24hr by default (Gamma schema field).
    If Gamma returns 422, retries once without order/ascending.
    """
    session = requests.Session()
    events: List[Dict[str, Any]] = []
    offset = 0

    while True:
        base_params = {
            "active": "true",
            "closed": "false",
            "limit": limit,
            "offset": offset,
        }

        # First try with ordering (nice-to-have)
        params = dict(base_params)
        if order:
            params["order"] = order
            params["ascending"] = ascending

        url = f"{GAMMA_BASE}/events"

        backoff = 1.0
        last_err = None

        for attempt in range(max_retries):
            try:
                batch = _get_json(session, url, params)
                break
            except requests.HTTPError as e:
                last_err = e
                status = getattr(e.response, "status_code", None)

                # If 422, retry WITHOUT order/ascending (most common cause)
                if status == 422 and ("order" in params or "ascending" in params):
                    params = dict(base_params)  # strip ordering
                    try:
                        batch = _get_json(session, url, params)
                        break
                    except Exception as e2:
                        last_err = e2

                # retry on 429 / transient 5xx
                if status in (429, 500, 502, 503, 504):
                    time.sleep(backoff)
                    backoff = min(backoff * 2.0, 16.0)
                    continue

                raise
            except Exception as e:
                last_err = e
                time.sleep(backoff)
                backoff = min(backoff * 2.0, 16.0)
        else:
            raise RuntimeError(f"Request failed after {max_retries} retries: {last_err}")

        if not batch:
            break
        if not isinstance(batch, list):
            raise RuntimeError(f"Unexpected /events response type: {type(batch)}; payload={str(batch)[:500]}")

        events.extend(batch)

        if len(batch) < limit:
            break

        offset += limit

    return events


def flatten_markets_from_events(events: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    flat: List[Dict[str, Any]] = []

    for ev in events:
        ev_id = ev.get("id")
        ev_slug = ev.get("slug")
        ev_title = ev.get("title")
        ev_category = ev.get("category")
        ev_start = ev.get("startDate")
        ev_end = ev.get("endDate")

        markets = ev.get("markets") or []
        if not isinstance(markets, list):
            continue

        for m in markets:
            outcomes = _maybe_parse_json_string(m.get("outcomes"))
            prices = _maybe_parse_json_string(m.get("outcomePrices"))

            outcome_map = None
            if isinstance(outcomes, list) and isinstance(prices, list) and len(outcomes) == len(prices):
                try:
                    outcome_map = {str(outcomes[i]): float(prices[i]) for i in range(len(outcomes))}
                except Exception:
                    outcome_map = None

            flat.append({
                "market_id": m.get("id"),
                "condition_id": m.get("conditionId"),
                "slug": m.get("slug") or ev_slug,
                "question": m.get("question"),
                "category": m.get("category") or ev_category,
                "active": m.get("active"),
                "closed": m.get("closed"),
                "enable_order_book": m.get("enableOrderBook"),
                "start_date": m.get("startDate") or ev_start,
                "end_date": m.get("endDate") or ev_end,
                "liquidity": m.get("liquidity"),
                "volume": m.get("volume"),
                "volume_24hr": m.get("volume24hr"),  # event uses volume24hr too :contentReference[oaicite:1]{index=1}
                "outcomes": outcomes,
                "outcome_prices": prices,
                "outcome_price_map": outcome_map,
                "event": {
                    "event_id": ev_id,
                    "event_slug": ev_slug,
                    "event_title": ev_title,
                },
                "raw": m,
            })

    return flat


def export_active_polymarket_json(limit: int = 100) -> Dict[str, Any]:
    events = fetch_all_active_events(limit=limit)
    markets = flatten_markets_from_events(events)
    return {
        "metadata": {
            "generated_at": _now_iso(),
            "event_count": len(events),
            "market_count": len(markets),
            "source": "polymarket-gamma",
            "events_endpoint": f"{GAMMA_BASE}/events?active=true&closed=false",
        },
        "events": events,
        "markets": markets,
    }


if __name__ == "__main__":
    data = export_active_polymarket_json(limit=100)
    with open("polymarket_active_markets.json", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"Exported {data['metadata']['event_count']} events and {data['metadata']['market_count']} markets")

Exported 7729 events and 47158 markets


## Polymarket DB Export — Tests

Smoke-tests for `src/data/poly/poly_fetch.py` and `src/data/db_export_poly_markets.py`.


In [4]:
# Test 1: poly_fetch — fetch one page of events and flatten into markets
from src.data.poly.poly_fetch import iter_active_event_batches, flatten_markets_from_events

event_batch = next(iter(iter_active_event_batches(limit=10)))
markets = flatten_markets_from_events(event_batch)

print(f"Events in page : {len(event_batch)}")
print(f"Markets flattened: {len(markets)}")
assert len(event_batch) > 0, "Expected at least one event"
assert len(markets) > 0, "Expected at least one market"

# Inspect the first market
m = markets[0]
print("\n--- First market ---")
for k in ("condition_id", "market_id", "question", "category", "active",
          "volume", "volume_24hr", "liquidity",
          "outcome_yes_price", "outcome_no_price", "event_id", "event_title"):
    print(f"  {k}: {m.get(k)}")

print("\nTest 1 passed ✓")


Events in page : 10
Markets flattened: 585

--- First market ---
  condition_id: 0x3bed62b0b7e3eb52c1f0d8a5d11edad1f74989038fc1cae2889cdbe96a248dfe
  market_id: 1294628
  question: US strikes Iran by February 5, 2026?
  category: None
  active: True
  volume: 4461531.494124
  volume_24hr: None
  liquidity: None
  outcome_yes_price: 0.0
  outcome_no_price: 1.0
  event_id: 114242
  event_title: US strikes Iran by...?

Test 1 passed ✓


In [5]:
# Test 2: normalize_market — verify all expected columns are present and types are right
from src.data.db_export_poly_markets import normalize_market

EXPECTED_KEYS = {
    "condition_id", "market_id", "slug", "question", "category",
    "active", "closed", "enable_order_book",
    "start_date", "end_date",
    "liquidity", "volume", "volume_24hr",
    "outcome_yes_price", "outcome_no_price", "outcome_prices",
    "event_id", "raw",
}

rows_with_cid = [m for m in markets if m.get("condition_id")]
assert rows_with_cid, "No markets with condition_id found — check Gamma API response"

row = normalize_market(rows_with_cid[0])
missing = EXPECTED_KEYS - set(row.keys())
assert not missing, f"normalize_market is missing keys: {missing}"

# Numeric fields should be float or None, never a raw string
for num_col in ("liquidity", "volume", "volume_24hr", "outcome_yes_price", "outcome_no_price"):
    val = row[num_col]
    assert val is None or isinstance(val, float), \
        f"{num_col} should be float|None, got {type(val)}: {val!r}"

# raw must be a JSON string (not a dict)
import json
assert isinstance(row["raw"], str), "raw should be a JSON string"
json.loads(row["raw"])  # ensure it's valid JSON

print("Normalised row sample:")
for k, v in row.items():
    if k != "raw":
        print(f"  {k}: {v!r}")

print("\nTest 2 passed ✓")


Normalised row sample:
  condition_id: '0x3bed62b0b7e3eb52c1f0d8a5d11edad1f74989038fc1cae2889cdbe96a248dfe'
  market_id: '1294628'
  slug: 'us-strikes-iran-by-february-5-2026'
  question: 'US strikes Iran by February 5, 2026?'
  category: None
  active: True
  closed: True
  enable_order_book: True
  start_date: '2026-01-29T22:21:59.421469Z'
  end_date: '2026-02-05T00:00:00Z'
  liquidity: None
  volume: 4461531.494124
  volume_24hr: None
  outcome_yes_price: 0.0
  outcome_no_price: 1.0
  outcome_prices: '{"Yes": 0.0, "No": 1.0}'
  event_id: '114242'

Test 2 passed ✓


In [6]:
# Test 3: full streaming pipeline dry-run (no DB) — count markets across all pages
from src.data.poly.poly_fetch import iter_flat_market_batches
from src.data.db_export_poly_markets import normalize_market

total = 0
skipped = 0
seen_events = set()

for batch in iter_flat_market_batches(limit=100):
    for m in batch:
        if not m.get("condition_id"):
            skipped += 1
            continue
        normalize_market(m)  # must not raise
        total += 1
        if m.get("event_id"):
            seen_events.add(m["event_id"])

print(f"Total markets normalised : {total}")
print(f"Skipped (no condition_id): {skipped}")
print(f"Unique events            : {len(seen_events)}")

assert total > 0, "Expected at least one normalised market"
print("\nTest 3 (full pipeline dry-run) passed ✓")


Total markets normalised : 47536
Skipped (no condition_id): 21
Unique events            : 7014

Test 3 (full pipeline dry-run) passed ✓


In [1]:
print('hello world')

hello world
